In [ ]:
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
import os

import warnings
warnings.filterwarnings('ignore')

  
importance_list=[]
first_ord=[]
score_list = []
pvalue_list =[]
mse_list = []
mae_list = []
ypred=[]
ytest=[]

import json

with open('E:/newGRASS/VARStxt/configAMPSIT.json') as config_file:
    config = json.load(config_file)

totalhours = config['totalhours']
variables = config['variables']
regions = config['regions']
verticalmax = config['verticalmax']
totalsim = config['totalsim']
parameter_names = config['parameter_names']
output_path = config['output_pathname']
tun_iter = config['tun_iter']
    
#############################################################################################         

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr

def sa_randomforest(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun):
  from sklearn.ensemble import RandomForestRegressor
  from joblib import dump, load

  if tun==1:

    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score

    rf = RandomForestRegressor()

    #params = {
    #    'n_estimators': (10, 100),  
    #    'max_depth': (2, 20),  
    #    'min_samples_split': (2, 10), 
    #    'min_samples_leaf': (1, 10),  
    #    'max_features': ['sqrt', 'log2', None],  
    #    'bootstrap': [True, False],  
    #}

  
    params = {
        'n_estimators': (10, 30),  
        'max_depth': (2, 6),  
        'min_samples_split': (2, 10), 
        'min_samples_leaf': (1, 8),  
        'max_features': [None],  
        'bootstrap': [True],  
    }


    opt = BayesSearchCV(rf, params, n_iter=tun_iter, cv=5, n_jobs=-1)
    opt.fit(X_train, y_train)
    
    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_rf_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_rf_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_rf_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))


    best_params = opt.get_params()
    
    best_rf_params = {key.replace("estimator__", ""): value for key, value in best_params.items() if key.startswith('estimator__')}
    rf = RandomForestRegressor(**best_rf_params)

    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_test)

    importances = rf.feature_importances_

    dump(rf, output_path+'rf_model_'+f[:-4]+'.joblib')
    
  else:
    
    if tun==2:
      loaded_model = load(output_path+'rf_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = loaded_model.feature_importances_
      
    else:            

      rf = RandomForestRegressor(n_estimators=20, max_depth=5, max_features='log2',min_samples_leaf= 1, min_samples_split= 2)
    
      rf.fit(X_train, y_train)
      
      y_pred = rf.predict(X_test)

      importances = rf.feature_importances_


  if len(importance_list) <= totalhours-1:
      importance_list.append(importances)
  else:
      importance_list = []               
  
  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_test, y_pred)
  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_xgboost(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun):
  from xgboost import XGBRFRegressor
  from joblib import dump, load

  
  if tun==1:

    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score

    gb = XGBRFRegressor()

    params = {
        'n_estimators': (10, 30),
        'max_depth': (2, 6),
    }
    '''
    from skopt.space import Real, Integer, Categorical
    params = {
        'n_estimators': Integer(50, 100),  # Numero di alberi
        'max_depth': Integer(3, 10),  # Profondità massima di ciascun albero
        'learning_rate': Real(0.01, 0.3, prior='log-uniform'),  # Tasso di apprendimento
        'min_child_weight': Integer(1, 10),  # Minimo peso richiesto per un nodo
        'gamma': Real(0, 0.5),  # Minima riduzione della perdita necessaria per fare ulteriori partizioni
        'subsample': Real(0.5, 1.0),  # Frazione di esempi da usare per addestrare ciascun albero
        'colsample_bytree': Real(0.5, 1.0),  # Frazione di feature da usare per costruire ciascun albero
        'reg_lambda': Real(1e-2, 1e2, prior='log-uniform'),  # Regolarizzazione L2 sui pesi
        'reg_alpha': Real(1e-2, 1e2, prior='log-uniform'),  # Regolarizzazione L1 sui pesi
    }
    '''
    opt = BayesSearchCV(gb, params, n_iter=tun_iter, cv=5, n_jobs=-1)

    opt.fit(X_train, y_train)
    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_xgb_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_xgb_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_xgb_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    best_params = opt.get_params()
    gb = XGBRFRegressor(**best_params)

    gb.fit(X_train, y_train)
  
    y_pred = gb.predict(X_test)
    
    importances = gb.feature_importances_

    dump(gb, output_path+'xgb_model_'+f[:-4]+'.joblib')
    
  else:

    if tun==2:
      loaded_model = load(output_path+'xgb_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = loaded_model.feature_importances_
      
    else:        
    
      gb = XGBRFRegressor(n_estimators=20, max_depth=5, max_features='log2',min_samples_leaf= 1, min_samples_split= 2)
    
      gb.fit(X_train, y_train)
    
      y_pred = gb.predict(X_test)

      importances = gb.feature_importances_

  if len(importance_list) <= totalhours-1:
      importance_list.append(importances)
  else:
      importance_list = []               
  
  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_test, y_pred)
  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_cart(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun):
  from sklearn.tree import DecisionTreeRegressor
  
  from joblib import dump, load
  
  if tun==1:

    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score

    dt = DecisionTreeRegressor()

    params = {
        'max_depth': (2, 6),
        'min_samples_split': (2, 10),
        'min_samples_leaf': (1, 5)
    }

    opt = BayesSearchCV(dt, params, n_iter=tun_iter, cv=5, n_jobs=-1)

    opt.fit(X_train, y_train)
    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_cart_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_cart_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_cart_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    best_params = opt.get_params()
    
    dt_params = {key: value for key, value in best_params.items() if key in ['max_depth', 'min_samples_split', 'min_samples_leaf']}

    dt = DecisionTreeRegressor(**dt_params)

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)

    importances = dt.feature_importances_

    dump(dt, output_path+'cart_model_'+f[:-4]+'.joblib')
    
  else:
    
    if tun==2:
      loaded_model = load(output_path+'cart_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = loaded_model.feature_importances_
      
    else:    
    
      dt = DecisionTreeRegressor(max_depth=5,min_samples_leaf= 1, min_samples_split= 2)
    
      dt.fit(X_train, y_train)

      y_pred = dt.predict(X_test)

      importances = dt.feature_importances_


  if len(importance_list) <= totalhours-1:
      importance_list.append(importances)
  else:
      importance_list = []               
  
  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_test, y_pred)
  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]



def sa_lassoregression(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun):
  import numpy as np
  from sklearn.linear_model import LassoCV
  from joblib import dump, load
  
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score
    
    lasso_cv = LassoCV()

    params = {
        'eps': (1e-8, 1e-1, 'log-uniform'),
        'n_alphas': (50, 300),
        'tol': (1e-8, 1e-3, 'log-uniform'),
        'cv': [5, 7],
    }

    opt = BayesSearchCV(lasso_cv, params, n_iter=tun_iter, cv=5, n_jobs=-1)

    opt.fit(X_train, y_train)

    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_lasso_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_lasso_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_lasso_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    y_pred = opt.best_estimator_.predict(X_test)

    importances = np.abs(opt.best_estimator_.coef_)

    dump(opt, output_path+'lasso_model_'+f[:-4]+'.joblib')

  else:
  
    if tun==2:
      loaded_model = load(output_path+'lasso_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = np.abs(loaded_model.best_estimator_.coef_)
      
    else:
  
  
      lasso_cv = LassoCV(cv=5)
      
      lasso_cv.fit(X_train, y_train)                        

      y_pred = lasso_cv.predict(X_test)
      
      importances = np.abs(lasso_cv.coef_)
                      
  
       
  if len(importance_list) <= totalhours-1:
      #importance_list.append(importances)
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
  else:
      importance_list = []

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]

def sa_svm(X_train, X_test, y_train, y_test, problem, N, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun):
  from sklearn import svm
  from joblib import dump, load
  from sklearn.model_selection import train_test_split, cross_val_score 
  
  
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score
    
    svm_model = svm.SVR()
    
    #params = {
    #    'C': (1e-3, 1e+1, 'log-uniform'),
    #    'kernel': ['linear'],
    #    'gamma': (1e-3, 1e+1, 'log-uniform'),
    #    'epsilon': (1e-4, 1e-1, 'log-uniform'), 
    #}
    
    from skopt.space import Real, Categorical, Integer
    params = {
        'C': Real(1e-4, 1e+1, prior='log-uniform'), 
        'kernel': Categorical(['poly']),
        'gamma': Real(1e-3, 1e+1, prior='log-uniform'),
        'epsilon': Real(1e-4, 1e-1, prior='log-uniform'),
        'degree': Integer(2, 6),
        'coef0': Real(0, 10),
    }


    svr = BayesSearchCV(svm_model, params, n_iter=tun_iter, cv=5, n_jobs=-1)

    svr.fit(X_train, y_train)

    score = cross_val_score(svr, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_svm_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_svm_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_svm_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(svr.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    y_pred = svr.predict(X_test)
    #coef = svr.best_estimator_.coef_[0]  #classical linear
    
    #import shap
    #best_svm_model = svr.best_estimator_
    #explainer = shap.Explainer(best_svm_model.predict, X_train)
    #shap_values = explainer(X_test)
    #coef = np.abs(shap_values.values).mean(axis=0) #shap
    
    dump(svr, output_path+'svm_model_'+f[:-4]+'.joblib')
  
  else:
 
    if tun==2:
      svr = load(output_path+'svm_model_'+f[:-4]+'.joblib')
      
      y_pred = svr.predict(X_test)
      
      #coef=loaded_model.best_estimator_.coef_[0] #classical linear
      
    else:
      
      svr = svm.SVR(kernel='linear', C=1, epsilon=0.1)
      svr.fit(X_train, y_train)
      
      y_pred = svr.predict(X_test)
      
      #coef=clf.coef_[0] #classical linear
  
  
  from SALib.sample import sobol
  X_sobol = sobol.sample(problem, N, calc_second_order=True)
  from numpy import zeros
  Y_sobol = zeros((len(X_sobol), 1))
  for i in range(len(X_sobol)):
      Y_sobol[i] =svr.predict(X_sobol[i].reshape(1, -1))
  Y_sobol = Y_sobol.reshape(-1)
  from SALib.analyze import sobol
  Si = sobol.analyze(problem, Y_sobol, calc_second_order=True, print_to_console=False)
  importances = np.where(Si['ST'] < 1, Si['ST'], np.nan) #sobol
  
  
  if len(importance_list) <= totalhours-1:
      #importance_list.append(coef) #classical linear / shap
      #importance_list.append(importances) #sobol
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
  else:
      importance_list = []

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_baesyanreg(X_train, X_test, y_train, y_test,problem,N, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,first_ord,f,tun):
  from sklearn.model_selection import train_test_split

  from sklearn.linear_model import BayesianRidge
  from joblib import dump, load


  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score
    
    br_model = BayesianRidge()

    params = {
        'max_iter': (100, 500),  
        'tol': (1e-9, 1e-3, 'log-uniform'),
        'alpha_1': (1e-10, 1e-4, 'log-uniform'),  
        'alpha_2': (1e-10, 1e-4, 'log-uniform'),  
        'lambda_1': (1e-10, 1e-4, 'log-uniform'),  
        'lambda_2': (1e-10, 1e-4, 'log-uniform'), 
        'fit_intercept': [True, False], 
    }


    br = BayesSearchCV(br_model, params, n_iter=tun_iter, cv=5, n_jobs=-1)

    br.fit(X_train, y_train)

    score = cross_val_score(br, X_train, y_train, cv=5).mean()
    
    if os.path.exists(f'{output_path}tuning_results_br_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_br_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_br_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(br.best_params_))
        file.write("Cross-validation score: {}\n".format(score))
    
    dump(br, output_path+'br_model_'+f[:-4]+'.joblib')
    y_pred = br.predict(X_test)
    
  else:    
    if tun==2:
      br = load(output_path+'br_model_'+f[:-4]+'.joblib')
      y_pred = br.predict(X_test)
    else:
      br = BayesianRidge()
      br.fit(X_train, y_train)
      
      y_pred = br.predict(X_test)


  from SALib.sample import sobol
  X_sobol = sobol.sample(problem, N, calc_second_order=True)
  from numpy import zeros
  Y_sobol = zeros((len(X_sobol), 1))
  for i in range(len(X_sobol)):
      Y_sobol[i] = br.predict(X_sobol[i].reshape(1, -1))
  Y_sobol = Y_sobol.reshape(-1)
  from SALib.analyze import sobol
  Si = sobol.analyze(problem, Y_sobol, calc_second_order=True, print_to_console=False)

  importances = Si['ST']
  first_order = Si['S1']
  interactions=Si['S2']
  np.savetxt(output_path+f'interactions_matrix_{len(importance_list)}.txt', interactions, delimiter='\t', fmt='%f')
  
  
  if len(importance_list) <= totalhours-1:
      #importance_list.append(importances)
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
      first_ord.append(first_order)
  else:
      importance_list = []
      first_ord=[]


  from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value  
  
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[] 

def sa_gaussianreg(X_train, X_test, y_train, y_test,problem,N, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,first_ord,f,tun):

  from sklearn.gaussian_process import GaussianProcessRegressor
  from sklearn.gaussian_process.kernels import RBF
  from sklearn.gaussian_process.kernels import Matern
  from joblib import dump, load
  
  
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score  
    from skopt.space import Real, Categorical, Integer
    from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, Sum, Product
        
    #kernel = RBF()    
    #kernel = Matern()
    kernel = RationalQuadratic()
    #kernel = Sum(Matern(), RationalQuadratic())
    #kernel = Product(Matern(), RationalQuadratic())
    gp_model = GaussianProcessRegressor(kernel=kernel)
    
    '''
    ##RBF
    #params = {
    #'alpha': (1e-10, 1e+3, 'log-uniform'),  
    #'n_restarts_optimizer': (0, 20),
    #'kernel__length_scale': (1e-2, 1e+2, 'log-uniform'),
    #}
    params = {
        'alpha': Real(1e-8, 1e+3, prior='log-uniform'),
        'n_restarts_optimizer': Integer(0, 20),
        'kernel__length_scale': Real(1e-2, 1e+2, prior='log-uniform'),
    }    
    '''
    ''
    ##rational quadratic
    params = {
    'alpha': (1e-10, 1e+3, 'log-uniform'),  
    'n_restarts_optimizer': (0, 20),
    'kernel__length_scale': (1e-2, 1e+2, 'log-uniform'),
    'kernel__alpha': Real(1e-2, 1e+2, prior='log-uniform'),
    }
    '''
    ''
    ##matern
    params = {
    'alpha': (1e-10, 1e+3, 'log-uniform'),  
    'n_restarts_optimizer': (0, 20),
    'kernel__length_scale': (1e-2, 1e+2, 'log-uniform'),
    'kernel__nu': Real(1e-2, 1e+2, prior='log-uniform'),
    }    
    '''

    ##product/sum
    #params = {
    #'alpha': Real(1e-10, 1e+3, prior='log-uniform'),
    #'n_restarts_optimizer': Integer(0, 20),
    #'kernel__k1__length_scale': Real(1e-2, 1e+2, prior='log-uniform'),  # Per Matern nel kernel Sum/Product
    #'kernel__k2__length_scale': Real(1e-2, 1e+2, prior='log-uniform'),  # Per RationalQuadratic nel kernel Sum/Product
    #'kernel__k1__nu': Real(0.5, 2.5),  # Parametro aggiuntivo per Matern
    #'kernel__k2__alpha': Real(0.1, 10),  # Parametro aggiuntivo per RationalQuadratic
    #}
    
    gp = BayesSearchCV(gp_model, params, n_iter=tun_iter, cv=5,n_jobs=-1)
    gp.fit(X_train, y_train)

    score = cross_val_score(gp, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_gp_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_gp_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_gp_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(gp.best_params_))
        file.write("Cross-validation score: {}\n".format(score))
    
    dump(gp, output_path+'gp_model_'+f[:-4]+'.joblib')
    
    y_pred = gp.predict(X_test)
        
  else:  
    
    if tun==2:
      gp = load(output_path+'gp_model_'+f[:-4]+'.joblib')
      y_pred = gp.predict(X_test)
    else:
      kernel = RBF(length_scale=1.0) ##########################################################
      gp = GaussianProcessRegressor(kernel=kernel)
      gp.fit(X_train, y_train)
      
      y_pred = gp.predict(X_test)
  
  
  from SALib.sample import sobol
  X_sobol = sobol.sample(problem, N, calc_second_order=True)
  from numpy import zeros
  Y_sobol = zeros((len(X_sobol), 1))
  for i in range(len(X_sobol)):
      Y_sobol[i] = gp.predict(X_sobol[i].reshape(1, -1))
  Y_sobol = Y_sobol.reshape(-1)
  from SALib.analyze import sobol
  Si = sobol.analyze(problem, Y_sobol, calc_second_order=True, print_to_console=False)

  importances = np.where(Si['ST'] < 1, Si['ST'], np.nan)
  first_order=Si['S1']
  interactions=Si['S2']
  np.savetxt(output_path+f'interactions_matrix_{len(importance_list)}.txt', interactions, delimiter='\t', fmt='%f')
  
  if len(importance_list) <= totalhours-1:
      #importance_list.append(importances)
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
      first_ord.append(first_order)
  else:
      importance_list = []
      first_ord=[]

  from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
  
  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value  
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]

  

#############################################################################################
import ipywidgets as widgets
from IPython.display import display

def create_predict_and_plot_global(meth,N,var,hpoint,vpoint,hour,tun, n_sample, figsize=(10,6)):
  def predict_and_plot(**x_dict):

    if var >= 1 and var <= len(variables):
        nam1 = variables[var - 1]
    else:
        nam1 = 'Invalid var value'

    if hpoint >= 1 and hpoint <= len(regions):
        nam2 = regions[hpoint - 1]
    else:
        nam2 = 'Invalid hpoint value'        

    name=nam1+'_'+nam2+'_lev'+str(vpoint)
    file_list=[nam1+'_'+nam2+'_lev'+str(vpoint)+'_'+str(i)+'.txt' for i in range(1,totalhours+1)]

    Xnonscaled = np.loadtxt(output_path+'X.txt') 


    for file in file_list:

      f=file
      ynonscaled = np.loadtxt(output_path+file, delimiter=',')
      
      
      Xnonscaled = Xnonscaled[:N, :]
      ynonscaled = ynonscaled[:N]

      min_len = min(len(Xnonscaled), len(ynonscaled))
      Xnonscaled = Xnonscaled[:min_len, :]
      ynonscaled = ynonscaled[:min_len]
           
      y=ynonscaled
      X=Xnonscaled              

      from sklearn.model_selection import train_test_split
      
      partition=0.3
        
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=partition, random_state=42)

      from sklearn.preprocessing import StandardScaler
      from joblib import dump

      scalerX = StandardScaler()
      X_train = scalerX.fit_transform(X_train)
      X_test = scalerX.transform(X_test)

      scalery = StandardScaler()
      y_train = scalery.fit_transform(y_train.reshape(-1, 1)).ravel()
      y_test = scalery.transform(y_test.reshape(-1, 1)).ravel()
      
      ''
      Xlow = np.min(X_train, axis=0)
      Xup = np.max(X_train, axis=0)
      Nn = n_sample 
      bounds = [(Xlow[i], Xup[i]) for i in range(Xlow.shape[0])]
      problem = {'num_vars': X.shape[1], 'names': parameter_names, 'bounds': bounds}            
      ''

      if meth==1:
        Si = sa_randomforest(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun)
        method='randomforest'
      elif meth==2:
        Si = sa_lassoregression(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun)
        method='lasso'
      elif meth==3:
        Si = sa_svm(X_train, X_test, y_train, y_test,problem,Nn, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun)
        method='svm'
      elif meth==4:        
        Si = sa_baesyanreg(X_train, X_test, y_train, y_test,problem,Nn, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,first_ord,f,tun)
        method='br'
      elif meth==5:
        Si = sa_gaussianreg(X_train, X_test, y_train, y_test,problem,Nn, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,first_ord,f,tun)
        method='gp'
      elif meth==6:
        Si = sa_xgboost(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun)
        method='xgboost'              
      elif meth==7:
        Si = sa_cart(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,f,tun)
        method='cart'
        
        
      data = np.array([score_list, pvalue_list, mse_list, mae_list]).T
      
      import pandas as pd
      df = pd.DataFrame(data, columns=['score', 'pvalue' , 'mse', 'mae'])
      importance_df = pd.DataFrame(importance_list)
      if hour==totalhours:
        importance_df.to_csv(output_path+'importance'+method+str(N)+file[:-7]+'.txt', header=False, index=False, sep=' ')
        df.to_csv(output_path+'df'+method+str(N)+file[:-7]+'.txt', header=False, index=False, sep=' ')

#############################################################################################
    import matplotlib
    import matplotlib.pyplot as plt

    matplotlib.rc('xtick',labelsize=10)
    matplotlib.rc('ytick',labelsize=10)
    
    x=np.arange(totalhours)

    pvalue = df.loc[hour-1, 'pvalue']
    if pvalue < 0.0001:
        pvalue_str = "<0.0001"
    else:
        pvalue_str = "{:.3g}".format(pvalue)

    
    font=12
    
    if meth in [4,5]:
    
      fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 12))

      x = np.arange(totalhours)
      
      from matplotlib.cm import get_cmap
      cmap = get_cmap('viridis')
      colors = [cmap(i / (len(parameter_names)-1)) for i in range(len(parameter_names))]
      #colors = ['b','g','r','c','m','y']
      #colors = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#999999']
      for i in range(len(parameter_names)):     
        axs[0,0].plot(x,[abs(importance_list[j][i]) for j in range(totalhours)],label=parameter_names[i],color=colors[i])  
      
      axs[0,0].set_xlabel("Time",fontsize=12)
      axs[0,0].set_ylabel("Importance",fontsize=12)
      axs[0,0].set_title('Importance time evolution',fontsize=12)
      axs[0,0].legend(prop={'size':10, 'weight':'normal'}, ncol=4, loc='upper center', bbox_to_anchor=(0.5, 0.99))
      axs[0,0].grid(True)

      ln1 =axs[0,1].plot(df.index, df['score'], color=colors[0], label='Score')
      ln2 =axs[0,1].plot(df.index, df['mse'], color=colors[1], label='MSE')
      ln3 =axs[0,1].plot(df.index, df['mae'], color=colors[2], label='MAE')
      twin=axs[0,1].twinx()
      ln4 =twin.plot(df.index, df['pvalue'], color=colors[3], label='p-value')
      twin.set_ylabel('p-value',fontsize=12)
      twin.set_ylim(0, 1e-15)
      axs[0,1].set_xlabel('Time',fontsize=12)
      axs[0,1].set_ylabel('Value',fontsize=12)
      axs[0,1].set_title('Metrics time evolution',fontsize=12)
      axs[0,1].set_ylim(0, 1)
      lns = ln1 + ln2 + ln3 + ln4
      labels = [l.get_label() for l in lns]
      axs[0,1].legend(lns,labels,prop={'size':10,'weight':'normal'},loc='upper center', bbox_to_anchor=(0.15, 0.8))
      axs[0,1].grid(True)

      axs[1,0].scatter(ytest[hour-1], ypred[hour-1])
      ideal = [min(ytest[hour-1]), max(ytest[hour-1])]
      axs[1,0].plot(ideal, ideal, 'r--')
      axs[1,0].set_xlabel('True Values',fontsize=12)
      axs[1,0].set_ylabel('Predictions',fontsize=12)
      axs[1,0].set_title('Hour: {}; Performance (score: {:.2f} [p: {}], mse: {:.2f}, mae: {:.2f})'.format(hour-1, df.loc[hour-1, 'score'], pvalue_str, df.loc[hour-1, 'mse'], df.loc[hour-1, 'mae']))
      axs[1,0].grid(True)

      axs[1,1].bar(parameter_names,importance_list[hour-1])
      axs[1,1].set_xticklabels(parameter_names, rotation=90, ha='right')
      axs[1,1].set_xlabel("Features",fontsize=12)
      axs[1,1].set_ylabel("Coefficient",fontsize=12)
      axs[1,1].set_title('Hour: '+str(hour-1)+'; Sobol total index',fontsize=12)
      axs[1,1].grid(True)

      import seaborn as sns
      file_path = os.path.join(output_path, f'interactions_matrix_{hour-1}.txt')
      inter_mat = np.loadtxt(file_path, delimiter='\t')
      sns.heatmap(inter_mat, annot=True, cmap='coolwarm', fmt=".3f", cbar_kws={'label': 'Sobol second order index'},
                  xticklabels=parameter_names, yticklabels=parameter_names, ax=axs[2, 0])
      axs[2, 0].set_title(f'Interactions Matrix {hour-1}',fontsize=12)
      
      
      axs[2,1].bar(parameter_names,first_ord[hour-1])
      axs[2,1].set_xticklabels(parameter_names, rotation=90, ha='right')
      axs[2,1].set_xlabel("Features",fontsize=12)
      axs[2,1].set_ylabel("Coefficient",fontsize=12)
      axs[2,1].set_title('Hour: '+str(hour-1)+'; Sobol first order index',fontsize=12)    
      axs[2,1].grid(True)

    else:
      fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

      x = np.arange(totalhours)
      
      from matplotlib.cm import get_cmap
      cmap = get_cmap('viridis')
      
      
      colorsfill = plt.cm.tab10(np.linspace(0, 1, 6))
      cmap = get_cmap('viridis')
      colors = [cmap(i / (len(parameter_names)-1)) for i in range(len(parameter_names))]
      for i in range(len(parameter_names)):    
        axs[0,0].plot(x,[abs(importance_list[j][i]) for j in range(totalhours)],label=parameter_names[i],color=colors[i]) 
      
      axs[0,0].set_xlabel("Time",fontsize=12)
      axs[0,0].set_ylabel("Importance",fontsize=12)
      axs[0,0].set_title('Importance time evolution',fontsize=12)
      axs[0,0].set_ylim(0, 1)
      axs[0,0].legend(prop={'size':10, 'weight':'normal'}, ncol=4, loc='upper center', bbox_to_anchor=(0.5, 0.99))
      axs[0,0].grid(True)
      
      ln1 =axs[0,1].plot(df.index, df['score'], color=colors[0], label='Score')
      ln2 =axs[0,1].plot(df.index, df['mse'], color=colors[1], label='MSE')
      ln3 =axs[0,1].plot(df.index, df['mae'], color=colors[2], label='MAE')
      twin=axs[0,1].twinx()
      ln4 =twin.plot(df.index, df['pvalue'], color=colors[3], label='p-value')
      twin.set_ylabel('p-value',fontsize=12)
      twin.set_ylim(0, 1e-15)
      axs[0,1].set_xlabel('Time',fontsize=12)
      axs[0,1].set_ylabel('Value',fontsize=12)
      axs[0,1].set_title('Metrics time evolution',fontsize=12)
      axs[0,1].set_ylim(0, 1)
      lns = ln1 + ln2 + ln3 + ln4
      labels = [l.get_label() for l in lns]
      axs[0,1].legend(lns,labels,prop={'size':10,'weight':'normal'},loc='upper center', bbox_to_anchor=(0.15, 0.8))
      axs[0,1].grid(True)
      
      axs[1,0].scatter(ytest[hour-1], ypred[hour-1])
      ideal = [min(ytest[hour-1]), max(ytest[hour-1])]
      axs[1,0].plot(ideal, ideal, 'r--')
      axs[1,0].set_xlabel('True Values',fontsize=12)
      axs[1,0].set_ylabel('Predictions',fontsize=12)
      axs[1,0].set_title('Hour: {}; Performance (score: {:.2f} [p: {}], mse: {:.2f}, mae: {:.2f})'.format(hour-1, df.loc[hour-1, 'score'], pvalue_str, df.loc[hour-1, 'mse'], df.loc[hour-1, 'mae']))
      axs[1,0].grid(True)
      
      axs[1,1].bar(parameter_names,abs(importance_list[hour-1]))
      
      axs[1,1].set_xlabel("Features",fontsize=12)
      axs[1,1].set_xticklabels(parameter_names, rotation=90, ha='right')
      axs[1,1].set_ylabel("Coefficient",fontsize=12)
      axs[1,1].set_title('Hour: '+str(hour-1)+'; Feature importance',fontsize=12)
      axs[1,1].grid(True)
      
    fig.suptitle(method + ' on ' + name[:-9] + ' N:'+ str(N) + ' v:' + str(vpoint), fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path+ method+'_'+name[:-9] +'_hour'+str(hour-1)+ '_v'+str(vpoint)+'.png', dpi=300)
    plt.show()



  return predict_and_plot
########################################################################

def nearest_divisible(n , lst):
  size = len(lst)
  remainder = n % (size+2)
  if remainder == 0:
    return n
  diff = size - remainder
  return n + diff

n_sample_calc = nearest_divisible(1000, parameter_names)

choices=['vpoint','hpoint','variable','N_simul','method','hour','tuning']
vpointmin=1;
vpointmax=verticalmax;
hpointmin=1;
hpointmax=len(regions);
varmin=1;
varmax=len(variables);
Nmin=totalsim*1/10;
Nmax=totalsim;
methmin=1;
methmax=7; ###
tunmin=0;
tunmax=2;
hourmin=1;
hourmax=totalhours;
Xmin=[vpointmin,hpointmin,varmin,Nmin,methmin,hourmin,tunmin]
Xmax=[vpointmax,hpointmax,varmax,Nmax,methmax,hourmax,tunmax]
Nstep=[verticalmax-1,hpointmax-1,varmax-1, (Nmax - Nmin) // 10, methmax-1, totalhours-1 , tunmax]

import ipywidgets as widgets
from IPython.display import display, clear_output

def interactive_global(_, meth, N, var, hpoint, vpoint, hour, tun, with_button=False):
    Xnew = [vpoint, hpoint, var, N, meth, hour, tun]
    
    sliders = {}
    slider_widgets = []
    label_widgets = []

    for i, f in enumerate(choices):
        SliderClass = widgets.IntSlider if f in ['vpoint', 'hpoint', 'variable', 'N_simul', 'method', 'hour', 'tuning'] else widgets.FloatSlider
        slider = SliderClass(
            value=int(Xnew[i]) if SliderClass == widgets.IntSlider else Xnew[i],
            min=int(Xmin[i]) if SliderClass == widgets.IntSlider else Xmin[i],
            max=int(Xmax[i]) if SliderClass == widgets.IntSlider else Xmax[i],
            step=1 if SliderClass == widgets.IntSlider else (Xmax[i] - Xmin[i]) / Nstep[i],
            layout=widgets.Layout(width='400px', margin='0 0 0 40px')
        )
        sliders[f] = slider
        slider_widgets.append(slider)

    label_texts = [
        "<div style='width:240px; text-align:right;'><b>Vertical levels:</b> {}</div>".format(verticalmax),
        "<div style='width:240px; text-align:right;'><b>Domain horizontal point:</b><br>{}</div>".format(', '.join(regions)),
        "<div style='width:240px; text-align:right;'><b>Variable:</b><br>{}</div>".format(', '.join(variables)),
        "<div style='width:240px; text-align:right;'><b>Number of simulations:</b><br>Choose sample size</div>",
        "<div style='width:240px; text-align:right;'><b>Methods:</b><br>1=RF, ..., 7=CART</div>",
        "<div style='width:240px; text-align:right;'><b>Hour:</b><br>Select hour of interest</div>",
        "<div style='width:240px; text-align:right;'><b>Tuning:</b><br>0=Off, 1=On, 2=Load</div>"
    ]

    label_widgets = [widgets.HTML(txt) for txt in label_texts]

    status_output = widgets.Output()


    should_stop = {"value": False}

    stop_button = widgets.Button(
        description="STOP",
        layout=widgets.Layout(width='100px', height='45px'),
        tooltip="Interrupt the process",
        button_style='danger'
    )
    stop_button.style.font_weight = 'bold'
    stop_button.style.font_size = '16px'

    def stop_execution(_):
        should_stop["value"] = True
        status_output.clear_output()
        with status_output:
            display(widgets.HTML("<b style='color: red;'>Execution interrupted by user.</b>"))

    stop_button.on_click(stop_execution)


    run_button = None
    if with_button:
        run_button = widgets.Button(
            description="RUN",
            layout=widgets.Layout(width='180px', height='45px'),
            tooltip="Run the model with selected configuration"
        )
        run_button.style.button_color = '#0078D7'
        run_button.style.font_weight = 'bold'
        run_button.style.font_size = '16px'





        def update_results_within(_):
            should_stop["value"] = False  # reset stop flag
            status_output.clear_output()
            with status_output:
                display(widgets.HTML("<div style='color: green; font-weight: bold;'>Running model... Please wait.</div>"))

            # (slider values read here...)

            if tun == 1:
                with status_output:
                    display(widgets.HTML("<i style='color: orange;'>Tuning is enabled — this might take a while...</i>"))

            try:
                if should_stop["value"]:
                    raise RuntimeError("Process was interrupted by user.")

                on_change_global = create_predict_and_plot_global(
                    meth=meth, N=N, var=var, hpoint=hpoint, vpoint=vpoint, hour=hour, tun=tun, n_sample=n_sample_calc
                )

                if should_stop["value"]:
                    raise RuntimeError("Process was interrupted by user.")

                on_change_global()

                if should_stop["value"]:
                    raise RuntimeError("Process was interrupted by user.")

                with status_output:
                    status_output.clear_output()
                    display(widgets.HTML("<div style='color: blue; font-weight: bold;'>Output generated successfully.</div>"))

            except Exception as e:
                with status_output:
                    status_output.clear_output()
                    display(widgets.HTML(f"<b style='color: red;'>Error: {str(e)}</b>"))

        run_button.on_click(update_results_within)

    # Pair labels with sliders
    paired_rows = []
    for label, slider in zip(label_widgets, slider_widgets):
        row = widgets.HBox(
            [label, slider],
            layout=widgets.Layout(
                justify_content='flex-start',
                align_items='center',
                padding='10px',
                border='1px solid #ddd',
                margin='6px 0',
                border_radius='6px',
                background_color='#f9f9f9',
                gap='60px'
            )
        )
        paired_rows.append(row)

    sliders_and_labels_box = widgets.VBox(
        paired_rows,
        layout=widgets.Layout(
            justify_content='center',
            width='auto',
            padding='10px'
        )
    )

    # === Methods Legend ===
    methods_legend = widgets.HTML(
        """
        <div style="text-align: left; padding: 12px 20px; border: 1px solid #ccc; border-radius: 8px; background-color: #f4f4f4; font-size: 14px; line-height: 1.6; max-width: 280px;">
        <b>Available algorithms</b><br>
        (1) Random Forest<br>
        (2) Lasso<br>
        (3) Support Vector Regression<br>
        (4) Bayesian regression<br>
        (5) Gaussian regression<br>
        (6) XGBoost<br>
        (7) CART
        </div>
        """
    )

    legend_and_sliders_box = widgets.HBox(
        [sliders_and_labels_box, methods_legend],
        layout=widgets.Layout(
            justify_content='center',
            align_items='flex-start',
            gap='40px',
            width='100%'
        )
    )

    # === Headers ===
    title = widgets.HTML("""
        <div style='text-align:center; font-family:"Segoe UI", sans-serif; font-size: 38px; font-weight: bold; color: #2c3e50; margin-top: 15px;'>
            ML-AMPSIT
        </div>
    """)
    subtitle = widgets.HTML("""
        <div style='text-align:center; font-family:Arial; font-size: 18px; color: #555;'>
            Machine Learning-based Automated Multi-method Parameter Sensitivity and Importance analysis Tool
        </div>
    """)
    subtitle2 = widgets.HTML("""
        <div style='text-align:center; font-size: 14px; color: #888;'>
            Interactive UI by <b>Dario Di Santo</b>, Dept. of Civil, Environmental and Mechanical Engineering, University of Trento
        </div>
    """)
    subtitle3 = widgets.HTML("""
        <div style='text-align:center; font-size: 14px; color: #666;'>
            Configure the sliders and click RUN. The output will be saved as specified in <code>configAMPSIT.json</code>.<br>
            Project: <a href='https://github.com/ML-AMPSIT/ML-AMPSIT' target='_blank'>GitHub Repository</a>
        </div>
    """)

    # === Final UI ===
    ui = widgets.VBox([
        title, subtitle, subtitle2, subtitle3,
        widgets.HTML('<hr>'),
        legend_and_sliders_box,
        widgets.HTML('<br>'),
        widgets.HBox([run_button, stop_button], layout=widgets.Layout(justify_content='center', gap='20px')),
        widgets.HTML('<br>'),
        status_output
    ], layout=widgets.Layout(align_items='center'))

    return sliders, ui





on_change_global = create_predict_and_plot_global(
    meth=1, N=totalsim, var=1, hpoint=1, vpoint=1, hour=totalhours, tun=0, n_sample=n_sample_calc
)

sliders, ui = interactive_global(on_change_global, 1, totalsim, 1, 1, 1, totalhours, 0, with_button=True)
display(ui)

# -----------------------------------------------------------